Decumulate rain fields
======================
and write it back to GRIB, ensuring proper validity

In [1]:
import usevortex
import epygram
import os

Vortex 1.2.3 loaded ( Wednesday 30. May 2018, at 17:49:44 )


In [2]:
pwd = os.getcwd()
os.chdir('/home/mary/tmp')

In [3]:
vortex_description = dict(# Resource...
                          kind='gridpoint',
                          geometry='ncaled0025',
                          nativefmt='grib',
                          origin='historic',
                          model='arome',
                          vapp='arome',
                          vconf='caledonie',
                          cutoff='prod',
                          term=range(4),
                          date='2018050800',
                          # Provider...
                          namespace='vortex.multi.fr',
                          experiment='oper',
                          block='forecast',
                          # Container
                          shouldfly=True)

In [4]:
with usevortex.quiet_get():
    resources = usevortex.get_resources(getmode='epygram',
                                        **vortex_description)

# [2018/05/30-17:49:45][common.util.usepygram][<module>:0027][INFO]: Epygram 1.3.2 loaded.


In [5]:
rain = [r.readfield('indicatorOfParameter:150') for r in resources[1:]]  # term 0, no rain

In [6]:
for i in range(len(rain)):
    if i == 0: continue  # term 1, no decumulation to do
    rain[i].operation('-', rain[i-1])
    rain[i].validity[0].set(cumulativeduration=(rain[i].validity[0].get()-rain[i-1].validity[0].get()))

In [7]:
rain_series = epygram.formats.resource(os.path.join(pwd,'../outputs/rain_series.grib'), 'w', fmt='GRIB')
for r in rain:
    print(r.validity[0])
    rain_series.writefield(r)
rain_series.close()

FieldValidity containing:
    _basis: 2018-05-08 00:00:00
    _date_time: 2018-05-08 01:00:00
    _statistical_process_on_duration: None
    _cumulativeduration: 1:00:00
FieldValidity containing:
    _basis: 2018-05-08 00:00:00
    _date_time: 2018-05-08 02:00:00
    _statistical_process_on_duration: None
    _cumulativeduration: 1:00:00
FieldValidity containing:
    _basis: 2018-05-08 00:00:00
    _date_time: 2018-05-08 03:00:00
    _statistical_process_on_duration: None
    _cumulativeduration: 1:00:00
